In [3]:
from ase.io import read
from npl.core import Nanoparticle as NP
from npl.descriptors import EnvironmentalTopologies as ETOP
import numpy as np

system = read("/home/riccardo/Scientific_Projects/PtAu_bigNps/300K/10_perc/run2/glo002.xyz")
system = NP.from_atoms(system)

%timeit ETOP().create(system)


137 ms ± 1.18 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [4]:
%timeit ETOP().update(system)

2.72 ms ± 120 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [49]:
class ExchangeMove:
    def __init__(self, system):
        self.occupation_matrix = system._occupation_matrix
        self.a = np.where(self.occupation_matrix==1)[1]
        self.b = np.where(self.occupation_matrix==0)[1]

    def exchange_move(self):
        swap1 = np.random.choice(self.a)
        swap2 = np.random.choice(self.b)

        self.a[swap1], self.b[swap2] = self.a[swap2], self.b[swap1]

        self.occupation_matrix[0,swap1], self.occupation_matrix[1,swap1] = self.occupation_matrix[1,swap1], self.occupation_matrix[0,swap1]
        self.occupation_matrix[0,swap2], self.occupation_matrix[1,swap2] = self.occupation_matrix[1,swap2], self.occupation_matrix[0,swap2]

ex = ExchangeMove(system)
ex.exchange_move()

In [50]:
for _ in range(20000):
    ex.exchange_move()
    ETOP().update(system)

In [20]:
np.where(system._occupation_matrix==1)[1]

array([   0,    1,    3, ..., 1460, 1461, 1462])